In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 19.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [3]:
from catboost import CatBoostClassifier

import pickle

with open(data_path + 'model/step1_SMOTE.dat', 'rb') as fp:
    step_1 = pickle.load(fp)
with open(data_path + 'model/step2_OPTUNA.dat', 'rb') as fp:
    step_2 = pickle.load(fp)
with open(data_path + 'model/step3_Op_SMOTE.dat', 'rb') as fp:
    step_3 = pickle.load(fp)
    scaler = pickle.load(fp)
with open(data_path + 'model/step4_model.dat', 'rb') as fp:
    step_4 = pickle.load(fp)

In [4]:
x = pd.read_parquet(data_path + 'processed/train_all.parquet')
y = pd.read_csv(data_path + 'data/one_hot_encoding.csv')

In [5]:
x.drop(['ID', '기준년월'], axis=1, inplace=True)

In [6]:
x_scaled = scaler.transform(x)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 310 entries, 이용금액_R3M_신용체크 to 변동률_RV평잔
dtypes: float64(32), int16(75), int32(68), int8(135)
memory usage: 1.8 GB


In [ ]:
y_test.value_counts()

A  B  C  D  E
0  0  0  0  1    384253
         1  0     69969
      1  0  0     25532
1  0  0  0  0       201
0  1  0  0  0        45
Name: count, dtype: int64

In [8]:
from sklearn.metrics import precision_recall_curve

def get_threshold(y_true, y_proba):
    # precision, recall, thresholds
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)

    # F1 계산
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)

    # 가장 높은 F1을 주는 threshold
    best_idx = f1_scores.argmax()
    best_threshold = thresholds[best_idx]
    best_f1 = f1_scores[best_idx]

    print(f"Best threshold: {best_threshold:.3f}")
    print(f"Best F1: {best_f1:.3f}, Precision: {precision[best_idx]:.3f}, Recall: {recall[best_idx]:.3f}")
    return best_threshold

In [34]:
y_proba = pd.DataFrame()

In [10]:
proba_1 = step_1.predict_proba(x_test)

In [12]:
e_threshold = get_threshold(y_test['E'], proba_1[:,1])

Best threshold: 0.642
Best F1: 0.923, Precision: 0.889, Recall: 0.961


In [37]:
y_proba['E'] = (proba_1[:,1] >= e_threshold).astype(int)

In [ ]:
y_proba

,E
0,1
1,1
2,1
3,1
4,0
...,...
479995,1
479996,1
479997,1
479998,1


In [36]:
mask = (y_proba['E'] == 0)
print(mask.sum())
X_train2 = x_test[mask]
X_train2.shape

17684


(17684, 310)

In [15]:
proba_2 = step_2.predict_proba(X_train2)[:,1]

In [16]:
mask = y_proba['E'] == 0
col = pd.Series(0, index=y_proba.index)
col[mask] = proba_2

/tmp/ipython-input-16-2852826554.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.02462716 0.0271026  0.09707335 ... 0.03407302 0.06384832 0.15372924]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  col[mask] = proba_2


In [17]:
y_ab = y_test['A'] + y_test['B']

In [18]:
ab_threshold = get_threshold(y_ab, col)

Best threshold: 0.723
Best F1: 0.612, Precision: 0.618, Recall: 0.606


In [19]:
col1 = pd.Series(0, index=y_proba.index)
col2 = pd.Series(0, index=y_proba.index)
col1[mask] = (proba_2 >= ab_threshold).astype(int)
col2[mask] = (proba_2 < ab_threshold).astype(int)
X_train3 = x_test[col2 == 1]
X_train4 = x_test[col1 == 1]

In [20]:
proba_3 = step_3.predict_proba(X_train3)[:,1]

In [21]:
col = pd.Series(0, index=y_proba.index)
mask = col2 == 1
col[mask] = proba_3
d_threshold = get_threshold(y_test['D'], col)

Best threshold: 0.320
Best F1: 0.436, Precision: 0.500, Recall: 0.387


/tmp/ipython-input-21-269303999.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.7130433  0.60055913 0.39602286 ... 0.63096553 0.5818984  0.34816718]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  col[mask] = proba_3


In [ ]:
mask.sum()

np.int64(0)

In [22]:
colD = pd.Series(0, index=y_proba.index)
colC = pd.Series(0, index=y_proba.index)

In [23]:
mask = col2 == 1
colD[mask] = (proba_3 >= d_threshold).astype(int)
colC[mask] = (proba_3 < d_threshold).astype(int)

In [24]:
proba_4 = step_4.predict_proba(X_train4)[:,1]

In [25]:
col = pd.Series(0, index=y_proba.index)
mask = col1 == 1
col[mask] = proba_4
b_threshold = get_threshold(y_test['B'], col)

Best threshold: 0.271
Best F1: 0.361, Precision: 0.688, Recall: 0.244


/tmp/ipython-input-25-1626228259.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04803634 0.02895986 0.01518072 0.03341187 0.04675502 0.04369693
 0.01546915 0.00452087 0.12764333 0.02197158 0.02890753 0.18251746
 0.02629254 0.06959488 0.11523474 0.08433004 0.00367224 0.03159906
 0.00793564 0.07293316 0.04650049 0.03769318 0.05854873 0.04084209
 0.00601083 0.00408072 0.00738134 0.12328144 0.05684623 0.03501978
 0.53129916 0.02117614 0.0244742  0.02215898 0.01503177 0.05637642
 0.02046008 0.01726383 0.04614993 0.032473   0.05919553 0.0130273
 0.02199745 0.01510388 0.02651151 0.10451574 0.01454477 0.01772624
 0.17381793 0.01294558 0.02347275 0.02973269 0.03412737 0.19523349
 0.02257743 0.05277753 0.05337606 0.01408185 0.03153127 0.0175165
 0.40493186 0.01943074 0.05080664 0.03901346 0.77581513 0.42373461
 0.01562353 0.31410821 0.07138175 0.05216953 0.11582122 0.14432963
 0.20852378 0.71391505 0.03747

In [26]:
mask = col1 == 1
colB = pd.Series(0, index=y_proba.index)
colA = pd.Series(0, index=y_proba.index)
colB[mask] = (proba_4 >= b_threshold).astype(int)
colA[mask] = (proba_4 < b_threshold).astype(int)

In [27]:
y_proba['A'] = colA
y_proba['B'] = colB
y_proba['C'] = colC
y_proba['D'] = colD

In [29]:
y_proba.value_counts()

E  A  B  C  D
1  0  0  0  0    415500
0  0  0  0  1     54090
         1  0     10169
   1  0  0  0       225
   0  1  0  0        16
Name: count, dtype: int64

In [30]:
y_decoded = y_proba.idxmax(axis=1)
y_true = y_test.idxmax(axis=1)

In [31]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_decoded)
cm

array([[   128,      2,     70,      1,      0],
       [     8,     11,     23,      3,      0],
       [    80,      3,   8333,  12475,   4641],
       [     8,      0,   1301,  27052,  41608],
       [     1,      0,    442,  14559, 369251]])

In [32]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_decoded, average='macro')
f1

0.5575888080836529

In [33]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_decoded, average='micro')
f1

0.84328125